### Load Library

# Original Code can be found in [here](https://github.com/floraxhuang/Movie-Recommendation-System)

In [1]:
pip install fastFM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for fastFM: filename=fastFM-0.2.10-cp310-cp310-linux_x86_64.whl size=214508 sha256=57c99bb78bd9141b07acc6340ea3e0b7df30b0cc823fed25172b45acd75d8257
  Stored in directory: /root/.cache/pip/wheels/93/92/52/2da7997fcb7a7ce9042ff3b33836ef0c2fd47aa95382d7a113
Successfully built fastFM
Note: you may need to restart the kernel to use updated packages.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens1m/ml-1m.test.negative
/kaggle/input/movielens1m/users.dat
/kaggle/input/movielens1m/ml-1m.test.csv
/kaggle/input/movielens1m/ml-1m.test.auc.csv
/kaggle/input/movielens1m/ratings.dat
/kaggle/input/movielens1m/README
/kaggle/input/movielens1m/ml-1m.test.rating
/kaggle/input/movielens1m/ml-1m.test_list.csv
/kaggle/input/movielens1m/ml-1m.train.csv
/kaggle/input/movielens1m/ml-1m.train.rating
/kaggle/input/movielens1m/movies.dat


In [3]:
#Load library
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import time
from math import sqrt
import random
import matplotlib as matplt
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, normalize
from fastFM import als
from sklearn.metrics import mean_absolute_error, mean_squared_error

from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate

### Load Data

In [4]:
#Load data
# path = "/Users/yh3093/Desktop/Personalization/Final Project/Data/"
path = "/kaggle/input/movielens1m/"

#Ratings
ratings = pd.read_csv(path+'ratings.dat', sep='::', header=None, engine='python')
ratings.columns = ['userId','movieId','rating','timestamp']
ratings = ratings.drop('timestamp', axis=1)

#Movies
# movies = pd.read_csv(path+'movies.dat', sep='::', header=None, engine='python')
movies = pd.read_csv(path+'movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
movies.columns = ['movieId','Title','Genres']

#Users
users = pd.read_csv(path+'users.dat', sep='::', header=None, engine='python')
users.columns = ['userId','Gender','Age','Occupation','Zip-code']
users = users.drop('Zip-code', axis=1)

### Data Quality

In [5]:
#Data quality
print('Duplicated rows in ratings file: ' + str(ratings.duplicated().sum()))

n_users = ratings.userId.unique().shape[0]
n_movies = ratings.movieId.unique().shape[0]

print('Number of users: {}'.format(n_users))
print('Number of movies: {}'.format(n_movies))
print('Sparsity: {:4.3f}%'.format(float(ratings.shape[0]) / float(n_users*n_movies) * 100))

Duplicated rows in ratings file: 0
Number of users: 6040
Number of movies: 3706
Sparsity: 4.468%


### Data Preprocessing

In [6]:
movies.Genres = movies.Genres.str.split('|')

In [7]:
def expand_df(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [8]:
ratings.shape

(1000209, 3)

In [9]:
movies = expand_df(movies, ['Genres'])
movies = movies.drop('Title', axis=1)

In [10]:
ratings = pd.merge(ratings, users, on="userId")

In [11]:
ratings_ffm = ratings.merge(movies, left_on='movieId', right_on='movieId', how='inner')

In [12]:
df_dummy = pd.get_dummies(ratings_ffm['Genres'])

In [13]:
df_new = pd.concat([ratings_ffm, df_dummy], axis=1)

In [14]:
df_final = df_new.groupby(["userId", "movieId", "rating", "Gender", "Age", "Occupation"])[df_new.columns.values[7:]].sum().reset_index()
print(df_final.shape)

(1000209, 24)


### Functions used in training factorization machine

In [15]:
#subset data
def subsetdata(data, by, subset_quantile):
    filter_standard = data.groupby([by]).size().reset_index(name='counts').counts.quantile(subset_quantile)
    subset_data = data.groupby(by).filter(lambda x: len(x) >= filter_standard)
    
    return filter_standard, subset_data

In [16]:
#split train and test data
def split_testtrain(ratings, fraction):
    #Transform data in matrix format
    colnames = ratings.columns.values
    new_colnames = ['1_user', '2_movie', '0_rating', '3_gender', '4_age', '5_occupation', 
                    '6_Action', '7_Adventure', '8_Animation', "9_Children's", '10_Comedy',
                    '11_Crime', '12_Documentary', '13_Drama', '14_Fantasy', '15_Film-Noir', 
                    '16_horror', '17_Musical', '18_Mystery', '19_Romance', '20_Sci-Fi', 
                    '21_Thriller', '22_War', '23_Western']
    ratings = ratings.rename(index=str, columns=dict(zip(colnames, new_colnames)))
    
    ratings_df = ratings.to_dict(orient="records")
    
    dv = DictVectorizer()
    ratings_mat = dv.fit_transform(ratings_df).toarray()
    
    #Split data
    x_train, x_test, y_train, y_test = train_test_split(ratings_mat[:,1:], ratings_mat[:,:1], test_size=fraction)
    
    return x_train, x_test, y_train.T[0], y_test.T[0]

In [17]:
#One hot encoding
def OneHotEncoding(train,test):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(train)
    train = encoder.transform(train)
    test = encoder.transform(test)
    return train, test

In [18]:
#Gridsearch for the optimal parameter
def param_selection(X, y, n_folds):
    start = time.time()
    grid_param = {  
    'n_iter' : np.arange(0,120,25)[1:],
    'rank' :  np.arange(2,12,4),
    }
    grid_search = GridSearchCV(als.FMRegression(l2_reg_w=0.1,l2_reg_V=0.1), cv=n_folds, param_grid=grid_param, verbose=10)
    grid_search.fit(X, y)
    grid_search.best_params_
    print(time.time()-start)
    return grid_search.best_params_

In [19]:
def rec_coverage(x_test, y_test, prediction, rec_num):
    ratings = pd.DataFrame()
    ratings['user'] = x_test[:,0]
    ratings['movie'] = x_test[:,1]
    ratings['rating'] = y_test
    
    pred = ratings.copy()
    pred['rating'] = prediction
    
    rating_table = pd.pivot_table(ratings, index='user', columns = 'movie', values = 'rating')
    pred_table = pd.pivot_table(pred, index='user', columns = 'movie', values = 'rating')
    
    rec_movies = []
    rec = pred_table - rating_table
    for user in rec.index:
            rec_item = pred_table.loc[user,:].sort_values(ascending = False).head(rec_num).index.tolist()
            rec_movies += rec_item
    n_rec = len(set(rec_movies))
    n_movies = pred_table.shape[1]
    coverage = round(float(n_rec)/n_movies,2)
    
    return coverage

In [20]:
def create_plot(x1, x2, x3, y1, y2, y3, kind):
    pal = sns.color_palette("Set2")
    
    matplt.figure.Figure(figsize=(5000,5000))
    plt.plot(x1, y1, c=pal[0], label="Filter-User", linewidth=3)
    plt.plot(x2, y2, c=pal[1], label="Filter-Movie", linewidth=3)
    plt.plot(x3, y3, c=pal[2], label="Filter-Both", linewidth=3)
    plt.legend(loc='best', fontsize=12)
    plt.xticks(fontsize=12);
    plt.yticks(fontsize=12);
    plt.xlabel("Sampled Data Size", fontsize=14);
    plt.ylabel(kind, fontsize=14);
    plt.title(kind, loc='center', fontsize=16);
    plt.show()

### Factorization Machine

In [21]:
def FieldFactorizationMachine(ratings, subset_by, subset_quantile, op_iter, op_rank):
    #Map value
    gender_dict = {"F": 0, "M": 1}
    ratings = ratings.replace({"Gender": gender_dict})
    
    #Initialize output
    final_output = pd.DataFrame()
    result_dict = []
    n_iteration = 1 
    last_RMSE = 100
    threshold = 0
    
    for quantile in subset_quantile:
        print("---Running iteration " + str(n_iteration) + " ---")
        print("---Subsetting Original Data---")
        
        #subset original data
        if subset_by == "user":
            filter_standard, subset_ratings = subsetdata(ratings, "userId", quantile)
        elif subset_by == "movie":
            filter_standard, subset_ratings = subsetdata(ratings, "movieId", quantile)
        else:
            f1, subset_u = subsetdata(ratings, "userId", quantile)
            f2, subset_ratings = subsetdata(subset_u, "movieId", quantile)
            filter_standard = "("+str(f1)+","+str(f2)+")"
        
        n_users = subset_ratings.userId.unique().shape[0]
        n_movies = subset_ratings.movieId.unique().shape[0]
        n_size = subset_ratings.shape[0]*subset_ratings.shape[1]
        
        sparsity = round(float(subset_ratings.shape[0]) / float(n_users*n_movies),2)
        
        print("---Spliting Test and Train Data---")
        #split test and train data
        xtrain, xtest, ytrain, ytest = split_testtrain(subset_ratings, 1/3)

        print("---Encoding Data---")
        #encode data
        xtrain_enc, xtest_enc = OneHotEncoding(xtrain, xtest)
        
        start = time.time()
        print("---Factorization Machine---")
        #Factorization machine
        fm = als.FMRegression(n_iter=op_iter, rank=op_rank, l2_reg_w=0.1, l2_reg_V=0.1)
        fm.fit(xtrain_enc, ytrain)
        predictions = fm.predict(xtest_enc)
        spent_time = time.time() - start
        #Evaluation metrics
        rmse = sqrt(mean_squared_error(ytest,predictions))
        mae = mean_absolute_error(ytest,predictions)
        coverage = rec_coverage(xtest, ytest, predictions, 10)
        
        if rmse < last_RMSE:
            last_RMSE = rmse
            threshold = filter_standard
            out = pd.DataFrame()
            out['user'] = xtest[:,0]
            out['movie'] = xtest[:,1]
            out['rating'] = ytest
            out['prediction'] = predictions
            final_output = out.copy()
        
        
        result_dict.append([quantile, filter_standard, n_size, n_users, n_movies, sparsity, op_iter, op_rank, spent_time, mae, rmse, coverage])
        n_iteration += 1
    
    results = pd.DataFrame(result_dict)
    results.columns = ["Quantile", "Threshold", "Size", "Num_Users", "Num_Movies", "Sparsity", "OP_Iter", "OP_Rank", "Running Time", "MAE", "RMSE", "Coverage"]
    
    final_output.to_csv("FFM_Output_"+subset_by+"_"+str(threshold)+".csv", sep=',', encoding='utf-8', index=False)
    
    return results

In [22]:
# quantile_list = np.arange(0.1,1,0.1)
quantile_list = np.array([0.8])
# quantile_list = np.array([1/3, 2/3])

In [23]:
k_values = [2, 3, 5]  # k의 값을 담은 리스트(k는 latent factor, rank)

### Subset method 1 - Subset data from less prolific users to prolific users

In [24]:
# 사용자 기반
accuracy_matrix_users = {}
for k in k_values:
    accuracy_matrix_users[k] = FieldFactorizationMachine(df_final, "user", quantile_list, 100, k)

/tmp/ipykernel_99/1186906881.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings.replace({"Gender": gender_dict})


---Running iteration 1 ---
---Subsetting Original Data---
---Spliting Test and Train Data---
---Encoding Data---
---Factorization Machine---


/tmp/ipykernel_99/1186906881.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings.replace({"Gender": gender_dict})


---Running iteration 1 ---
---Subsetting Original Data---
---Spliting Test and Train Data---
---Encoding Data---
---Factorization Machine---


/tmp/ipykernel_99/1186906881.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings.replace({"Gender": gender_dict})


---Running iteration 1 ---
---Subsetting Original Data---
---Spliting Test and Train Data---
---Encoding Data---
---Factorization Machine---


In [25]:
# accuracy_matrix_user

In [26]:
#size_norm_u = normalize(accuracy_matrix_user['Size'][:,np.newaxis], axis=0).ravel()
# size_norm_u = 1-np.array(accuracy_matrix_user['Quantile'])

In [27]:
# time_u = np.array(accuracy_matrix_user['Running Time'])
# mae_u = np.array(accuracy_matrix_user['MAE'])
# rmse_u = np.array(accuracy_matrix_user['RMSE'])
# coverage_u = np.array(accuracy_matrix_user['Coverage'])

In [28]:
accuracy_matrices_u = {}

# 사용자 기반
for k in k_values:
    size_norm_u = 1 - np.array(accuracy_matrix_users[k]['Quantile'])
    time_u = np.array(accuracy_matrix_users[k]['Running Time'])
    mae_u = np.array(accuracy_matrix_users[k]['MAE'])
    rmse_u = np.array(accuracy_matrix_users[k]['RMSE'])
    coverage_u = np.array(accuracy_matrix_users[k]['Coverage'])
    
    # 각 원소들의 평균값을 계산하여 다시 저장
    size_norm_u = size_norm_u
    time_u = time_u
    mae_u = mae_u
    rmse_u = rmse_u
    coverage_u = coverage_u
    
    accuracy_matrices_u[k] = [size_norm_u, time_u, mae_u, rmse_u, coverage_u]
#     print(accuracy_matrices_u[k])
#     print(accuracy_matrix_users[k])

In [29]:
for k in k_values:
    print(f"Latent Factor of {k} FM on user-based RMSE :",accuracy_matrices_u[k][3])

Latent Factor of 2 FM on user-based RMSE : [0.8678579]
Latent Factor of 3 FM on user-based RMSE : [0.86490412]
Latent Factor of 5 FM on user-based RMSE : [0.86961935]


### Subset method 2 - Subset data from less popular items to popular items

In [30]:
# 영화 기반
accuracy_matrix_movies = {}
for k in k_values:
    accuracy_matrix_movies[k] = FieldFactorizationMachine(ratings, "movie", quantile_list, 100, k)

/tmp/ipykernel_99/1186906881.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings.replace({"Gender": gender_dict})


---Running iteration 1 ---
---Subsetting Original Data---
---Spliting Test and Train Data---
---Encoding Data---
---Factorization Machine---


/tmp/ipykernel_99/1186906881.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings.replace({"Gender": gender_dict})


---Running iteration 1 ---
---Subsetting Original Data---
---Spliting Test and Train Data---
---Encoding Data---
---Factorization Machine---


/tmp/ipykernel_99/1186906881.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings.replace({"Gender": gender_dict})


---Running iteration 1 ---
---Subsetting Original Data---
---Spliting Test and Train Data---
---Encoding Data---
---Factorization Machine---


In [31]:
# accuracy_matrix_movie

In [32]:
#size_norm_m = normalize(accuracy_matrix_movie['Size'][:,np.newaxis], axis=0).ravel()
# size_norm_m = 1-np.array(accuracy_matrix_movie['Quantile'])

In [33]:
# time_m = np.array(accuracy_matrix_movie['Running Time'])
# mae_m = np.array(accuracy_matrix_movie['MAE'])
# rmse_m = np.array(accuracy_matrix_movie['RMSE'])
# coverage_m = np.array(accuracy_matrix_movie['Coverage'])

In [34]:
# 영화 기반
accuracy_matrices_m = {}

for k in k_values:
    size_norm_m = 1 - np.array(accuracy_matrix_movies[k]['Quantile'])
    time_m = np.array(accuracy_matrix_movies[k]['Running Time'])
    mae_m = np.array(accuracy_matrix_movies[k]['MAE'])
    rmse_m = np.array(accuracy_matrix_movies[k]['RMSE'])
#     print(rmse_m)
    coverage_m = np.array(accuracy_matrix_movies[k]['Coverage'])
    accuracy_matrices_m[k] = np.concatenate([size_norm_m, time_m, mae_m, rmse_m, coverage_m])
#     print(accuracy_matrices_m[k])

In [35]:
for k in k_values:
    print(f"Latent Factor of {k} FM on m-based RMSE :",accuracy_matrices_m[k][3])

Latent Factor of 2 FM on m-based RMSE : 0.8716999522041019
Latent Factor of 3 FM on m-based RMSE : 0.8749229784082948
Latent Factor of 5 FM on m-based RMSE : 0.8874443257028956


### Subset method 3 - Subset data in both user and item directions

In [36]:
# 사용자 + 영화 기반
accuracy_matrix_boths = {}
for k in k_values:
    accuracy_matrix_boths[k] = FieldFactorizationMachine(ratings, "both", quantile_list, 100, k)

/tmp/ipykernel_99/1186906881.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings.replace({"Gender": gender_dict})


---Running iteration 1 ---
---Subsetting Original Data---
---Spliting Test and Train Data---
---Encoding Data---
---Factorization Machine---


/tmp/ipykernel_99/1186906881.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings.replace({"Gender": gender_dict})


---Running iteration 1 ---
---Subsetting Original Data---
---Spliting Test and Train Data---
---Encoding Data---
---Factorization Machine---


/tmp/ipykernel_99/1186906881.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings.replace({"Gender": gender_dict})


---Running iteration 1 ---
---Subsetting Original Data---
---Spliting Test and Train Data---
---Encoding Data---
---Factorization Machine---


In [37]:
# accuracy_matrix_both = FieldFactorizationMachine(ratings, "both", quantile_list, 25, 2)

In [38]:
# accuracy_matrix_both

In [39]:
#size_norm_b = normalize(accuracy_matrix_both['Size'][:,np.newaxis], axis=0).ravel()
# size_norm_b = 1-np.array(accuracy_matrix_both['Quantile'])

In [40]:
# time_b = np.array(accuracy_matrix_both['Running Time'])
# mae_b = np.array(accuracy_matrix_both['MAE'])
# rmse_b = np.array(accuracy_matrix_both['RMSE'])
# coverage_b = np.array(accuracy_matrix_both['Coverage'])

In [41]:
# 사용자 + 영화 기반
accuracy_matrices_b = {}
    
for k in k_values:
    size_norm_b = 1 - np.array(accuracy_matrix_boths[k]['Quantile'])
    time_b = np.array(accuracy_matrix_boths[k]['Running Time'])
    mae_b = np.array(accuracy_matrix_boths[k]['MAE'])
    rmse_b = np.array(accuracy_matrix_boths[k]['RMSE'])
    coverage_b = np.array(accuracy_matrix_boths[k]['Coverage'])
    accuracy_matrices_b[k] = np.concatenate([size_norm_b, time_b, mae_b, rmse_b, coverage_b])
    
    # 각 원소들의 평균값을 계산하여 다시 저장
    size_norm_b = np.mean(size_norm_b)
    time_b = np.mean(time_b)
    mae_b = np.mean(mae_b)
    rmse_b = np.mean(rmse_b)
    coverage_b = np.mean(coverage_b)
    
    accuracy_matrices_b[k] = np.array([size_norm_b, time_b, mae_b, rmse_b, coverage_b])
    print(accuracy_matrices_b[k])

[ 0.2        12.04487658  0.66832785  0.85038863  0.52      ]
[ 0.2        15.35280609  0.66222868  0.84427688  0.61      ]
[ 0.2        21.13994122  0.65648756  0.83972343  0.7       ]


In [42]:
for k in k_values:
    print(f"Latent Factor of {k} FM on user-based RMSE :",accuracy_matrices_b[k][3])

Latent Factor of 2 FM on user-based RMSE : 0.8503886266755587
Latent Factor of 3 FM on user-based RMSE : 0.8442768809571605
Latent Factor of 5 FM on user-based RMSE : 0.8397234277622045


### Benchmark model - Collaborative Filtering Using k-Nearest Neighbors (kNN)

In [43]:
KNNdata = Dataset.load_builtin('ml-1m')

Dataset ml-1m could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /root/.surprise_data/ml-1m


In [44]:
algo = KNNBasic()

In [45]:
cross_validate(algo, KNNdata, measures = ['MAE','RMSE'], cv = 3, verbose = True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE, RMSE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MAE (testset)     0.7352  0.7341  0.7346  0.7346  0.0004  
RMSE (testset)    0.9315  0.9303  0.9313  0.9310  0.0005  
Fit time          23.15   23.20   23.21   23.19   0.03    
Test time         207.31  198.65  200.84  202.27  3.67    


{'test_mae': array([0.73518348, 0.73413523, 0.73457234]),
 'test_rmse': array([0.93145171, 0.930274  , 0.93125424]),
 'fit_time': (23.15263342857361, 23.201804399490356, 23.20963716506958),
 'test_time': (207.3052761554718, 198.65268850326538, 200.84406232833862)}

### Evaluation

In [46]:
# create_plot(size_norm_u, size_norm_m, size_norm_b, time_u, time_m, time_b, "Running Time")

In [47]:
# create_plot(size_norm_u, size_norm_m, size_norm_b, mae_u, mae_m, mae_b, "Mean Average Error")

In [48]:
# create_plot(size_norm_u, size_norm_m, size_norm_b, rmse_u, rmse_m, rmse_b, "Root Mean Square Error")

In [49]:
# create_plot(size_norm_u, size_norm_m, size_norm_b, coverage_u, coverage_m, coverage_b, "Coverage")

# Spec

In [50]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

In [51]:
print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

# CPU
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
cpu MHz		: 2200.216
cpu cores	: 2


In [52]:
print('# RAM')
run('cat /proc/meminfo | egrep "^MemTotal"')

# RAM
MemTotal:       32880784 kB


In [53]:
print('# OS')
run('uname -a')

# OS
Linux 692a89cb7831 5.15.133+ #1 SMP Tue Dec 19 13:14:11 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux
